In [ ]:
!pip install koreanize-matplotlib

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import geopandas as gpd
import plotly.express as px
from matplotlib.font_manager import FontProperties
from IPython.display import HTML as html_print
from termcolor import colored
from scipy import stats

In [ ]:
import pandas as pd

# 1. CSV 파일 불러오기
df = pd.read_csv('/content/owid-co2-data.csv')

# 2. 데이터 기본 확인
print("원본 데이터 형태:", df.shape)
print(df.head())
print(df.info())

# 3. 각 열의 결측치 개수 확인
missing_counts = df.isnull().sum()
print("열별 결측치 개수:")
print(missing_counts)

# 4. 결측치가 있는 행 제거 (행 기준으로 결측치 제거)
df_clean = df.dropna(subset=['year', 'country', 'co2'])

# 5. 'year' 열을 숫자형으로 변환
df_clean['year'] = pd.to_numeric(df_clean['year'], errors='coerce').astype(int)

# 6. 이상치 제거: 'co2' 열을 대상으로 IQR 기법 사용
Q1 = df_clean['co2'].quantile(0.25)
Q3 = df_clean['co2'].quantile(0.75)
IQR = Q3 - Q1
print("CO₂ Q1:", Q1, "Q3:", Q3, "IQR:", IQR)

# IQR 범위 내에 있는 데이터만 남김
df_clean = df_clean[(df_clean['co2'] >= Q1 - 1.5 * IQR) & (df_clean['co2'] <= Q3 + 1.5 * IQR)]

# 7. 전처리 후 데이터 확인
print("전처리 후 데이터 형태:", df_clean.shape)
print(df_clean.head())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

sns.set_theme(style="whitegrid", font="NanumGothic")
plt.rcParams['axes.unicode_minus'] = False

# 1. 데이터 불러오기 및 전처리
df = pd.read_csv('/content/owid-co2-data.csv')
df.columns = df.columns.str.strip()
df = df.dropna(subset=['year', 'country', 'co2'])
df['year'] = pd.to_numeric(df['year'], errors='coerce').astype(int)

# 2. 누적 국가별 CO₂ 배출량 계산 (전체 기간 합계)
df_total = df.groupby('country', as_index=False)['co2'].sum()

# 3. 대륙/소득 그룹 등 제외
exclude_list = [
    "World", "Asia", "Africa", "Europe", "North America", "South America",
    "Oceania", "European Union (27)", "European Union (28)",
    "High-income countries", "Upper-middle-income countries",
    "Lower-middle-income countries", "Low-income countries",
    "International transport",
    "Asia (excl. China & India)",
    "Europe (excl. EU-27)",
    "Europe (excl. EU-28)",
    "North America (excl. USA)"
]
df_total = df_total[~df_total['country'].isin(exclude_list)]

# 4. 상위 10개 국가 추출 (누적 배출량 기준)
df_top10 = df_total.nlargest(10, 'co2').copy()

# 5. 누적 배출량 비중 계산
total_emissions = df_total['co2'].sum()
df_top10['비중'] = df_top10['co2'] / total_emissions * 100

# 6. 파이 차트 시각화
plt.figure(figsize=(8,8))
plt.pie(
    df_top10['비중'],
    labels=df_top10['country'],  # 국가명만 표시
    autopct='%1.1f%%',
    startangle=90,
    colors=sns.color_palette('Reds_r', n_colors=10),
    wedgeprops=dict(edgecolor='white', linewidth=2)
)
plt.title("누적 국가별 CO₂ 배출량 비중", fontsize=18, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import matplotlib.font_manager as fm
import koreanize_matplotlib

# Colab에서 Noto Sans CJK 폰트 설치
!apt-get update -qq
!apt-get install fonts-noto-cjk -qq

# 시스템 폰트에서 Noto Sans CJK 찾기
font_paths = fm.findSystemFonts(fontpaths=None, fontext='ttf')
noto_font = ''
for fp in font_paths:
    if 'NotoSansCJK' in fp:
        noto_font = fp
        break

if not noto_font:
    print("Noto Sans 폰트를 찾지 못했습니다. 다른 폰트를 시도하세요.")
else:
    print("Noto Sans 폰트 경로:", noto_font)

# 1. 데이터 불러오기 및 전처리
df = pd.read_csv('/content/owid-co2-data.csv')
df = df.dropna(subset=['year', 'country', 'co2'])
df['year'] = pd.to_numeric(df['year'], errors='coerce').astype(int)

# 2. 누적 국가별 CO₂ 배출량 계산 (전체 연도의 합계)
emission_dict = df.groupby('country')['co2'].sum().to_dict()

# 3. 제외할 항목 (연합, 그룹 등)
exclude_list = [
    "World", "Asia", "Africa", "Europe", "North America", "South America",
    "Oceania", "European Union (27)", "European Union (28)",
    "High-income countries", "Upper-middle-income countries",
    "Lower-middle-income countries", "Low-income countries",
    "International transport",
    "Asia (excl. China & India)",
    "Europe (excl. EU-27)",
    "Europe (excl. EU-28)",
    "North America (excl. USA)"
]

# 4. 제외 목록에 해당하는 항목 제거 (영어 국가명을 기준으로)
emission_dict = {country: value for country, value in emission_dict.items() if country not in exclude_list}

# 5. 영어 → 한글 매핑 딕셔너리 (예시)
country_mapping = {
    "China": "중국",
    "United States": "미국",
    "India": "인도",
    "Russia": "러시아",
    "Japan": "일본",
    "Germany": "독일",
    "United Kingdom": "영국",
    "France": "프랑스",
    "South Korea": "한국",
    "North Korea": "북한",
    "Canada": "캐나다",
    "Brazil": "브라질",
    "Indonesia": "인도네시아",
    "Saudi Arabia": "사우디아라비아",
    "Malaysia": "말레이시아",
    "Poland": "폴란드",
    "Spain": "스페인",
    "Italy": "이탈리아",
    "Kazakhstan": "카자흐스탄",
    "Kyrgyzstan": "키르기스스탄",
    "Uzbekistan": "우즈베키스탄",
    "Egypt": "이집트",
    "Mexico": "멕시코",
    "Turkey": "튀르키예",
    "Pakistan": "파키스탄",
    "Qatar": "카타르",
    "Argentina": "아르헨티나",
    "South Africa": "남아프리카공화국",
    "Iraq": "이라크",
    "Algeria": "알제리",
    "Ukraine": "우크라이나",
    "Kuwait": "쿠웨이트",
    "Nepal": "네팔",
    "Angola": "앙골라",
    "Morocco": "모로코",
    "Libya": "리비아",
    "Chad": "차드",
    "United Arab Emirates": "아랍에미리트",
    "Myanmar": "미얀마",
    "Cameroon": "카메룬",
    "Iran": "이란",
    "Ethiopia": "에티오피아",
    "Tunisia": "튀니지",
    "Sudan": "수단",
    "Bahrain": "바레인",
    "Oman": "오만",
    "Syria": "시리아",
    "Lebanon": "레바논",
    "Jordan": "요르단",
    "Afghanistan": "아프가니스탄",
    "Mongolia": "몽골",
    "Thailand": "태국",
    "Vietnam": "베트남",
    "Laos": "라오스",
    "Cambodia": "캄보디아",
    "Philippines": "필리핀",
    "Sri Lanka": "스리랑카",
    "Singapore": "싱가포르",
    "Brunei": "브루나이",
    "Maldives": "몰디브",
    "Papua New Guinea": "파푸아뉴기니",
    "Australia": "오스트레일리아",
    "New Zealand": "뉴질랜드",
    "Taiwan": "대만",
    "Swden": "스웨덴",
    "Venezuela": "베네수엘라",
    # 필요 시 계속 추가...
}

# 6. 영어 국가명을 한글로 변환
emission_dict_kor = {}
for en_name, co2_val in emission_dict.items():
    kor_name = country_mapping.get(en_name, en_name)
    emission_dict_kor[kor_name] = co2_val

# 7. 워드클라우드 생성
wc = WordCloud(
    width=1600,
    height=800,
    background_color='white',
    colormap='Reds',
    font_path=noto_font if noto_font else None,
    max_words=300
)
wordcloud = wc.generate_from_frequencies(emission_dict_kor)

# 8. 결과 시각화
plt.figure(figsize=(20, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("누적 국가별 CO₂ 배출량", fontsize=26, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
import matplotlib.collections as mcoll
import matplotlib.colors as mcolors

# =========================================
# 1. 데이터 불러오기 및 전처리
# =========================================
# CSV 파일 불러오기
df = pd.read_csv('/content/owid-co2-data.csv')

# 필요한 열('year', 'country', 'co2')에 결측치 제거
required_cols = ['year', 'country', 'co2']
df_clean = df.dropna(subset=required_cols)

# 전 세계 데이터 구성: 'World' 값이 있다면 사용, 없으면 연도별 합계 계산
if 'World' in df_clean['country'].unique():
    df_world = df_clean[df_clean['country'] == 'World'].copy()
else:
    df_world = df_clean.groupby('year', as_index=False)['co2'].sum()

# 분석을 위해 0보다 큰 탄소배출량만 사용
df_world = df_world[df_world['co2'] > 0]
# 연도별 오름차순 정렬
df_world = df_world.sort_values('year').reset_index(drop=True)

# =========================================
# 2. 창의적인 시각화: 그라데이션 효과가 있는 면적 차트
# =========================================
# x: 연도, y: 탄소배출량
x = df_world['year'].values
y = df_world['co2'].values

# 점들을 선분으로 연결할 준비 (그라데이션 효과를 위해)
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

# x 값(연도)에 따라 색상이 변화하도록 정규화 및 colormap 설정
norm = plt.Normalize(x.min(), x.max())
lc = mcoll.LineCollection(segments, cmap='viridis', norm=norm, linewidth=3)
lc.set_array(x)

# 그래프 생성
fig, ax = plt.subplots(figsize=(12, 6))

# 그라데이션 선 추가
ax.add_collection(lc)
# 선의 범위를 업데이트하기 위해 투명 선을 그려줌
ax.plot(x, y, alpha=0)

# 선 아래 면적 채우기 (부드러운 느낌을 위해)
ax.fill_between(x, y, color='lightblue', alpha=0.3)

# 축 설정
ax.set_xlim(x.min(), x.max())
ax.set_ylim(y.min(), y.max())
ax.set_title('전 세계 탄소배출량 상승 추세 (시간에 따른 색상 변화)', fontsize=16, fontweight='bold')
ax.set_xlabel('연도', fontsize=14)
ax.set_ylabel('탄소배출량 (메트릭 톤)', fontsize=14)
ax.grid(True, linestyle='--', alpha=0.6)

# 색상 범례(컬러바) 추가: 연도를 기준으로 색상이 변화함을 표시
cb = fig.colorbar(lc, ax=ax)
cb.set_label('연도', fontsize=12)

plt.show()


In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

# 1. 데이터 불러오기
df = pd.read_csv('/content/owid-co2-data.csv')

# 2. 필요한 열('year', 'iso_code', 'co2', 'country') 결측치 제거
df = df.dropna(subset=['year', 'iso_code', 'co2', 'country'])

# 3. 기간 필터링: 1750년부터 2020년까지 (원하는 연도 범위 설정)
df = df[(df['year'] >= 1750) & (df['year'] <= 2020)]

# 4. 국가별, 연도별 중복 데이터가 있을 경우 평균 CO₂ 배출량으로 집계
df_grouped = df.groupby(['year', 'iso_code', 'country'], as_index=False)['co2'].mean()

# 5. 극단값 완화를 위한 99% 분위수(quantile) 계산
#    예) 상위 1%를 초과하는 값은 모두 동일한 최댓값(색상)으로 처리
max_co2_99 = df_grouped['co2'].quantile(0.99)

# 6. 사용자 지정 색상 스케일 (5단계 예시: 분홍~빨강)
custom_scale = [
    "#fee0d2", "#fc9272", "#ef3b2c", "#cb181d", "#67000d"
]

# 7. Choropleth 시각화 (Plotly Express)
fig = px.choropleth(
    df_grouped,
    locations='iso_code',         # 국가 ISO 코드 열
    color='co2',                  # 색상으로 표시할 값 (CO₂ 배출량)
    hover_name='country',         # 마우스 오버 시 표시할 국가명
    animation_frame='year',       # 연도별 애니메이션 슬라이더 생성
    color_continuous_scale=custom_scale,
    # range_color=(0, df_grouped['co2'].max())  # 실제 최댓값 기준 (극단값이 있으면 옅게 보일 수 있음)
    range_color=(0, max_co2_99),   # 상위 1%를 잘라낸 색상 범위 설정
    title='전 세계 CO₂ 배출량 (1750년 ~ 2020년)'
)

# 8. 컬러바(색상 범례) 설정
tick_vals = [0, max_co2_99*0.25, max_co2_99*0.5, max_co2_99*0.75, max_co2_99]
tick_text = [f"{val:,.0f}" for val in tick_vals]

fig.update_layout(
    margin=dict(l=0, r=0, t=50, b=0),
    coloraxis_colorbar=dict(
        title='CO₂ 배출량 (톤)',
        tickvals=tick_vals,
        ticktext=tick_text
    )
)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

#############################################
# 1. 데이터 불러오기 및 전처리
#############################################
df = pd.read_csv('/content/owid-co2-data.csv')
df.columns = df.columns.str.strip()
df = df.dropna(subset=['year', 'country'])
df['year'] = pd.to_numeric(df['year'], errors='coerce').astype(int)

# "World" 행 제외 (누적 배출량을 모든 국가 합산으로 계산하기 위함)
df = df[df['country'] != 'World']

# 분석할 섹터 리스트
sectors = ['coal_co2', 'oil_co2', 'gas_co2', 'cement_co2', 'flaring_co2', 'other_industry_co2']

# 필요한 열만 선택하여 결측치를 0으로 채움
df_sectors = df[sectors].fillna(0)

# 전체 기간(누적) 동안의 각 부문별 배출량 합계 계산
sector_sum = df_sectors.sum()

#############################################
# 2. 영어 섹터명을 한글로 변환
#############################################
sector_mapping = {
    'coal_co2': '석탄',
    'oil_co2': '석유',
    'gas_co2': '천연가스',
    'cement_co2': '시멘트',
    'flaring_co2': '플래어링',
    'other_industry_co2': '기타 산업'
}

sector_kor = []
emissions_val = []
for eng_name, val in sector_sum.items():
    kor_name = sector_mapping.get(eng_name, eng_name)
    if val > 0:
        sector_kor.append(kor_name)
        emissions_val.append(val)

# DataFrame 구성
df_sector = pd.DataFrame({
    '부문': sector_kor,
    '배출량': emissions_val
})

#############################################
# 3. Plotly로 파이 차트 생성 (인터랙티브)
#############################################
fig = px.pie(
    data_frame=df_sector,
    names='부문',
    values='배출량',
    color='부문',  # 부문별 색상 구분
    color_discrete_sequence=px.colors.sequential.Reds[::-1],  # 색상 팔레트 (진한 빨강부터 연하게)
    title='전체 기간 누적 부문별 CO₂ 배출량 비중 (모든 국가 합산)',
)

fig.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

# Seaborn 테마와 한글 폰트 설정
sns.set_theme(style="whitegrid", font="NanumGothic")
plt.rcParams['axes.unicode_minus'] = False

#############################################
# 1. 데이터 불러오기 및 전처리
#############################################
df = pd.read_csv('/content/owid-co2-data.csv')
df.columns = df.columns.str.strip()
df = df.dropna(subset=['year', 'country', 'co2'])
df['year'] = pd.to_numeric(df['year'], errors='coerce').astype(int)

# 분석 대상 연도: 2020년 (원하는 연도로 수정 가능)
target_year = 2020
df_year = df[df['year'] == target_year].copy()

# "World" 및 대륙/소득 그룹 등 실제 국가가 아닌 항목 제거
exclude_list = [
    "World",
    "Asia", "Asia (excl. China & India)",
    "Africa",
    "Europe", "Europe (excl. EU-27)", "Europe (excl. EU-28)",
    "North America", "North America (excl. USA)",
    "South America",
    "Oceania",
    "European Union (27)", "European Union (28)",
    "High-income countries", "Upper-middle-income countries",
    "Lower-middle-income countries", "Low-income countries",
    "International transport"
]
df_year = df_year[~df_year['country'].isin(exclude_list)].copy()

#############################################
# 2. 선진국, 개발도상국, 후진국 분류 (예시)
#############################################
advanced_countries = [
    "United States", "Canada", "Japan", "Germany", "United Kingdom", "France",
    "South Korea", "Australia", "New Zealand", "Switzerland", "Sweden",
    "Norway", "Denmark", "Finland", "Italy", "Spain", "Netherlands", "Austria", "Belgium"
]

developing_countries = [
    "China", "India", "Brazil", "Russia", "Mexico", "Indonesia", "Turkey",
    "South Africa", "Argentina", "Saudi Arabia", "Iran", "Thailand", "Malaysia",
    "Colombia", "Philippines"
]

underdeveloped_countries = [
    "Pakistan", "Bangladesh", "Nigeria", "Ethiopia", "DR Congo", "Afghanistan",
    "Nepal", "Uganda", "Tanzania", "Sudan", "Mozambique", "Burundi"
]

def classify_country(country):
    if country in advanced_countries:
        return "선진국"
    elif country in developing_countries:
        return "개발도상국"
    elif country in underdeveloped_countries:
        return "후진국"
    else:
        return "기타"

df_year["개발수준"] = df_year["country"].apply(classify_country)

#############################################
# 3. "기타" 그룹 제외
#############################################
df_year = df_year[df_year["개발수준"] != "기타"].copy()

#############################################
# 4. 그룹별 CO₂ 배출량 집계
#############################################
group_emissions = df_year.groupby("개발수준")["co2"].sum().reset_index()

#############################################
# 5. 시각화
#############################################
# (A) 막대 차트: 그룹별 CO₂ 배출량
plt.figure(figsize=(8,6))
sns.barplot(data=group_emissions, x='개발수준', y='co2', palette="coolwarm")
plt.title(f'개발수준별 CO₂ 배출량 ', fontsize=18, fontweight='bold')
plt.xlabel('개발수준', fontsize=14)
plt.ylabel('CO₂ 배출량 (톤)', fontsize=14)
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# (B) 파이 차트: 그룹별 CO₂ 배출량 비중
plt.figure(figsize=(8,8))
colors = sns.color_palette("coolwarm", n_colors=len(group_emissions))
plt.pie(
    group_emissions['co2'],
    labels=group_emissions['개발수준'],
    autopct='%1.1f%%',
    startangle=90,
    colors=colors
)
plt.title(f'개발수준별 CO₂ 배출량 비중 ', fontsize=18, fontweight='bold')
plt.tight_layout()
plt.show()

print('중국, 인도가 개발도상국으로 분류되어있어 탄소배출량이 선진국 보다 높다. 인구와도 관련이 있다.');
print('그럼 반드시 선진국이라고 배출이 아닌 인구수가 중요한게 아닐까');
print('GDP도 연관이 있겠지만 재생에너지도 연관이 있을수 있다.');
print('다만, 1인당 탄소 배출량은 단순히 인구수만으로 결정되는 것이 아니라, 해당 국가의 경제 구조, 에너지 효율, 산업화 수준, 기술 발전, 환경 정책 등 여러 요인에 의해 달라진다.');

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

sns.set_theme(style="whitegrid", font="NanumGothic")
plt.rcParams['axes.unicode_minus'] = False

# 1. OWID CO₂ 데이터 불러오기 및 전처리
df = pd.read_csv('/content/owid-co2-data.csv')
df.columns = df.columns.str.strip()

# 'World' 행이 있으면 그 데이터를 사용, 없으면 국가별 합산 (여기서는 'World' 행 사용)
if 'World' in df['country'].unique():
    df_world = df[df['country'] == 'World'].copy()
else:
    df_world = df.groupby('year', as_index=False)['co2'].sum()

# 연도와 co2 열을 숫자형으로 변환 및 정렬
df_world['year'] = pd.to_numeric(df_world['year'], errors='coerce')
df_world = df_world.dropna(subset=['year'])
df_world['year'] = df_world['year'].astype(int)
df_world = df_world.sort_values('year')

# 1750년부터 2020년까지의 데이터만 사용 (OWID 데이터에 따라 조정)
df_world = df_world[(df_world['year'] >= 1750) & (df_world['year'] <= 2020)]
df_world.rename(columns={'year': 'Year', 'co2': 'CO2_Emissions'}, inplace=True)

# 1970년과 2020년의 CO₂ 배출량을 이용하여 지난 50년간의 증가율 계산
co2_1970 = df_world[df_world['Year'] == 1970]['CO2_Emissions'].values[0]
co2_2020 = df_world[df_world['Year'] == 2020]['CO2_Emissions'].values[0]
perc_increase = ((co2_2020 - co2_1970) / co2_1970) * 100

# 2. 시대별 구간 정의 (실제 데이터 범위에 맞게 조정)
periods = [
    {"label": "산업혁명 전후 (1750-1900)", "start": 1750, "end": 1900, "color": "#a6cee3"},
    {"label": "급격한 산업화 (1900-1970)", "start": 1900, "end": 1970, "color": "#1f78b4"},
    {"label": "글로벌화 및 현대 (1970-2020)", "start": 1970, "end": 2020, "color": "#b2df8a"}
]

# 3. 시각화
fig, ax = plt.subplots(figsize=(16, 8))

# 실제 CO₂ 배출량 추세선을 그리기 (실제 데이터)
ax.plot(df_world['Year'], df_world['CO2_Emissions'], color='green', marker='o', markersize=4, linewidth=2, label='CO₂ 배출량')

# 시대별 구간을 영역으로 표시
for era in periods:
    ax.axvspan(era["start"], era["end"], color=era["color"], alpha=0.3)
    # 해당 구간의 중앙에 라벨 배치
    mid = (era["start"] + era["end"]) / 2
    ax.text(mid, ax.get_ylim()[1]*0.95, era["label"], ha='center', va='top', fontsize=14, fontweight='bold')

# 축 레이블 및 기타 스타일 설정
ax.set_xlabel("연도", fontsize=16, fontweight='bold')
ax.set_ylabel("CO₂ 배출량 (톤)", fontsize=16, fontweight='bold')
ax.grid(True, linestyle='--', alpha=0.7)
plt.xticks(fontsize=14, rotation=45)
plt.yticks(fontsize=14)

# 제목에 1970~2020년 증가율 포함
plt.title(f"1750~2020 전 세계 탄소 배출량 증가의 시대별 변화\n(산업혁명 이후 약 {perc_increase:.1f}% 증가)", fontsize=18, fontweight='bold')
ax.legend(fontsize=14, loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

sns.set_theme(style="whitegrid", font="NanumGothic")
plt.rcParams['axes.unicode_minus'] = False

# 요인과 기여 비율 (예시)
labels = ['화력발전소(화석연료 사용)', '자동차 등 교통수단', '축산업/농업(메탄 배출)', '열대 우림 사막화', '농경지 화학비료']
sizes = [40, 30, 15, 10, 5]
colors = ['#d62728', '#ff7f0e', '#1f77b4', '#2ca02c', '#9467bd']

plt.figure(figsize=(8,8))
plt.pie(
    sizes,
    labels=labels,
    autopct='%1.1f%%',
    startangle=90,
    colors=colors,
    wedgeprops=dict(edgecolor='white', linewidth=2)  # 각 조각 경계선을 흰색, 두께 2로 강조
)
plt.title("지구온난화에 영향을 미치는 주요 요인 기여도", fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

sns.set_theme(style="whitegrid", font="NanumGothic")
plt.rcParams['axes.unicode_minus'] = False

# ------------------------------
# 1. OWID CO₂ 데이터 불러오기 및 전처리
# ------------------------------
df_co2 = pd.read_csv('/content/owid-co2-data.csv')
df_co2.columns = df_co2.columns.str.strip()

# 'World' 행이 있으면 이를 사용, 없으면 국가별 합산
if 'World' in df_co2['country'].unique():
    df_global_co2 = df_co2[df_co2['country'] == 'World'].copy()
else:
    df_global_co2 = df_co2.groupby('year', as_index=False)['co2'].sum()

df_global_co2['year'] = pd.to_numeric(df_global_co2['year'], errors='coerce').astype(int)
df_global_co2 = df_global_co2.sort_values('year')
# 분석 범위: 1990~2020년
df_global_co2 = df_global_co2[(df_global_co2['year'] >= 1990) & (df_global_co2['year'] <= 2020)]
df_global_co2.rename(columns={'year': 'Year', 'co2': 'CO2_Emissions'}, inplace=True)

# ------------------------------
# 2. 재생에너지 데이터 불러오기 및 전처리 (03 modern-renewable-prod.csv)
# ------------------------------
df_renew = pd.read_csv('/content/03 modern-renewable-prod.csv')
df_renew.columns = df_renew.columns.str.strip()
df_renew = df_renew.dropna(subset=['Entity', 'Year'])
df_renew['Year'] = pd.to_numeric(df_renew['Year'], errors='coerce').astype(int)

# 재생에너지 관련 열 (열 이름은 실제 데이터에 맞춰 조정)
energy_cols = [
    'Electricity from wind (TWh)',
    'Electricity from hydro (TWh)',
    'Electricity from solar (TWh)',
    'Other renewables including bioenergy (TWh)'
]
for col in energy_cols:
    df_renew[col] = pd.to_numeric(df_renew[col], errors='coerce').fillna(0)

df_renew['Total Renewable (TWh)'] = df_renew[energy_cols].sum(axis=1)

# 'World' 행이 있으면 이를 사용, 없으면 국가별 합산
if 'World' in df_renew['Entity'].unique():
    df_global_renew = df_renew[df_renew['Entity'] == 'World'].copy()
else:
    df_global_renew = df_renew.groupby('Year', as_index=False)['Total Renewable (TWh)'].sum()

df_global_renew = df_global_renew.sort_values('Year')
# 분석 범위: 1990~2020년
df_global_renew = df_global_renew[(df_global_renew['Year'] >= 1990) & (df_global_renew['Year'] <= 2025)]

# ------------------------------
# 3. 이중 y축 그래프 시각화
# ------------------------------
fig, ax1 = plt.subplots(figsize=(14,8))

# 왼쪽 y축: 전 세계 CO₂ 배출량 (톤)
color_co2 = "#d62728"
ax1.plot(df_global_co2['Year'], df_global_co2['CO2_Emissions'],
         marker='o', linestyle='-', linewidth=2, color=color_co2, label='전 세계 CO₂ 배출량')
ax1.set_xlabel('연도', fontsize=14, fontweight='bold')
ax1.set_ylabel('CO₂ 배출량 (톤)', fontsize=14, fontweight='bold', color=color_co2)
ax1.tick_params(axis='y', labelcolor=color_co2)
ax1.tick_params(axis='x', rotation=45, labelsize=12)

# 오른쪽 y축: 전 세계 재생에너지 생산량 (TWh)
ax2 = ax1.twinx()
color_renew = "#1f77b4"
ax2.plot(df_global_renew['Year'], df_global_renew['Total Renewable (TWh)'],
         marker='s', linestyle='-', linewidth=2, color=color_renew, label='전 세계 재생에너지 생산량')
ax2.set_ylabel('재생에너지 생산량 (TWh)', fontsize=14, fontweight='bold', color=color_renew)
ax2.tick_params(axis='y', labelcolor=color_renew)
ax2.tick_params(axis='x', labelsize=12)

# 범례 병합
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=12)

plt.title("1990~2020 전 세계 CO₂ 배출량 및 재생에너지 생산량 추세", fontsize=16, fontweight='bold')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

지구온난화


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import koreanize_matplotlib

# 1. 데이터 불러오기 및 전처리
df = pd.read_csv('/content/GlobalLandTemperaturesByCountry.csv')

# 컬럼 확인
print("데이터 컬럼:", df.columns.tolist())
# 예상: ['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'Country']

# 'dt' 컬럼을 datetime 형식으로 변환하고 연도 추출
df['dt'] = pd.to_datetime(df['dt'], errors='coerce')
df['year'] = df['dt'].dt.year

# 결측치 제거: 평균 기온
df = df.dropna(subset=['AverageTemperature'])

# 2. 전 세계 평균 기온 추세 (라인 차트)
# 전 세계 데이터: 'Country'가 'World'인 경우가 아니라면, 모든 국가의 평균을 사용
df_global = df.groupby('year', as_index=False)['AverageTemperature'].mean()
df_global = df_global.sort_values('year').reset_index(drop=True)

plt.figure(figsize=(12,6))
sns.lineplot(data=df_global, x='year', y='AverageTemperature', marker='o', color='red')
plt.title('전 세계 평균 기온 추세', fontsize=16, fontweight='bold')
plt.xlabel('연도', fontsize=14)
plt.ylabel('평균 기온 (°C)', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

# 3. 전 세계 온난화 추세 (회귀 추세선 포함)
X = df_global[['year']]
y = df_global['AverageTemperature']
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)

plt.figure(figsize=(12,6))
sns.lineplot(data=df_global, x='year', y='AverageTemperature', marker='o', color='red', label='실제 평균기온')
plt.plot(df_global['year'], y_pred, color='blue', label='추세선')
plt.title('전 세계 평균 기온 추세', fontsize=16, fontweight='bold')
plt.xlabel('연도', fontsize=14)
plt.ylabel('평균 기온 (°C)', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()


print('기후 변동 원인')
print('엘니뇨,라니냐,화산분화,태양활동 등이 지구기온의 변동원인, 측정 방식,장비변화 및 오차, 데이터 불완전성 및 보정(과거의 데이터 품질 불량) ,장기추세+단기요인(연,계절,수년단위)')

# 4. 국가별 온도 상승 변화 (최초 vs. 최신 평균 기온 차이)
# 각 국가별로 연도별 평균 기온 계산
df_country = df.groupby(['Country', 'year'], as_index=False)['AverageTemperature'].mean()

# 각 국가별 첫 번째 기록과 마지막 기록을 추출
df_first = df_country.groupby('Country', as_index=False).first()
df_last = df_country.groupby('Country', as_index=False).last()

# 두 데이터프레임을 병합하여 온도 차이 계산
df_change = pd.merge(df_first, df_last, on='Country', suffixes=('_first', '_last'))
df_change['temp_change'] = df_change['AverageTemperature_last'] - df_change['AverageTemperature_first']

# 온도 상승이 가장 큰 상위 10개국 선택 (temp_change가 높은 순)
df_top10 = df_change.sort_values('temp_change', ascending=False).head(30)

plt.figure(figsize=(10,6))
sns.barplot(data=df_top10, x='temp_change', y='Country', palette='Reds_r')
plt.title('국가별 평균 기온 상승 변화 ', fontsize=16, fontweight='bold')
plt.xlabel('기온 상승 (°C)', fontsize=14)
plt.ylabel('국가', fontsize=14)
plt.grid(True, axis='x', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Colab에서 Plotly 그래프가 제대로 표시되도록 렌더러 설정
pio.renderers.default = "colab"

# -------------------------------------------------
# 1. Global Sea Level Rise 데이터 전처리
# -------------------------------------------------
df_sea = pd.read_csv('/content/Global_sea_level_rise.csv')
df_sea.columns = df_sea.columns.str.strip()  # 컬럼명 앞뒤 공백 제거
# 예상 컬럼: ['year', 'date', 'mmfrom1993-2008average']
df_sea.rename(columns={'mmfrom1993-2008average': 'sea_level'}, inplace=True)
df_sea['year'] = pd.to_numeric(df_sea['year'], errors='coerce')
df_sea = df_sea.dropna(subset=['year', 'sea_level']).sort_values('year')
# 해당 데이터셋의 최소 연도와 최대 연도
sea_min_year = int(df_sea['year'].min())
sea_max_year = int(df_sea['year'].max())

# -------------------------------------------------
# 2. Arctic Ice Extent 데이터 전처리
# -------------------------------------------------
df_arctic = pd.read_csv('/content/arctic_ice_extent.csv')
df_arctic.columns = df_arctic.columns.str.strip()
# 예상 컬럼: ['year', 'extent']
df_arctic.rename(columns={'extent': 'ice_extent'}, inplace=True)
df_arctic['year'] = pd.to_numeric(df_arctic['year'], errors='coerce')
df_arctic = df_arctic.dropna(subset=['year', 'ice_extent']).sort_values('year')
arctic_min_year = int(df_arctic['year'].min())
arctic_max_year = int(df_arctic['year'].max())

# -------------------------------------------------
# 3. Antarctic Mass 데이터 전처리
# -------------------------------------------------
df_antarctica = pd.read_csv('/content/Antartica_mass.csv')
df_antarctica.columns = df_antarctica.columns.str.strip()
# 예상 컬럼: ['year', 'Antarctic mass', 'Antarctic mass 1-sigma uncertainty']
df_antarctica.rename(columns={'Antarctic mass': 'ice_mass'}, inplace=True)
df_antarctica['year'] = pd.to_numeric(df_antarctica['year'], errors='coerce')
df_antarctica = df_antarctica.dropna(subset=['year', 'ice_mass']).sort_values('year')
anta_min_year = int(df_antarctica['year'].min())
anta_max_year = int(df_antarctica['year'].max())

# -------------------------------------------------
# 4. Plotly Subplots 생성 (각각 별도의 그래프)
# -------------------------------------------------
fig = make_subplots(rows=3, cols=1,
                    subplot_titles=("전 세계 해수면 상승 추세", "북극 빙하 면적 변화 추세", "남극 빙하 질량 변화 추세"))

# (A) 전 세계 해수면 상승 추세 (라인 차트)
fig.add_trace(
    go.Scatter(
        x=df_sea['year'],
        y=df_sea['sea_level'],
        mode='lines+markers',
        name='해수면 상승',
        line=dict(color='blue', width=3),
        marker=dict(size=8)
    ),
    row=1, col=1
)
fig.update_yaxes(title_text="해수면 상승 (mm)", row=1, col=1)
fig.update_xaxes(title_text="연도", row=1, col=1)

# (B) 북극 빙하 면적 변화 추세 (라인 차트)
fig.add_trace(
    go.Scatter(
        x=df_arctic['year'],
        y=df_arctic['ice_extent'],
        mode='lines+markers',
        name='빙하 면적',
        line=dict(color='cyan', width=3),
        marker=dict(size=8)
    ),
    row=2, col=1
)
fig.update_yaxes(title_text="빙하 면적 (백만 km²)", row=2, col=1)
fig.update_xaxes(title_text="연도", row=2, col=1)

# (C) 남극 빙하 질량 변화 추세 (라인 차트)
fig.add_trace(
    go.Scatter(
        x=df_antarctica['year'],
        y=df_antarctica['ice_mass'],
        mode='lines+markers',
        name='빙하 질량',
        line=dict(color='purple', width=3),
        marker=dict(size=8)
    ),
    row=3, col=1
)
fig.update_yaxes(title_text="빙하 질량 (GT)", row=3, col=1)
fig.update_xaxes(title_text="연도", row=3, col=1)

# 전체 레이아웃 업데이트
fig.update_layout(
    height=900,
    width=1000,
    title_text="지구온난화 관련 시계열 데이터",
    showlegend=False
)

fig.show()
print('빙하 면적은 우하향이 아닌 왔다갔다 한다. 이유는 계절적 요인과, 기후변동성이 있다. 겨울에 얼고, 여름에 언다. 엘니뇨, 라니냐등 여러 기후패턴으로 인해 변동이 있다.')

재생에너지

In [ ]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
df_3 = pd.read_csv("/content/03 modern-renewable-prod.csv")

In [ ]:
df_3.head()

In [ ]:
df_3.tail()

In [ ]:
def print_section_title(title):
    print(colored(title, 'blue', attrs=['bold', 'underline']))

def display_head_and_tail(dataframe, head=5):
    display(dataframe.head(head).style.set_caption("Head"))
    display(dataframe.tail(head).style.set_caption("Tail"))

def display_na(dataframe):
    na_df = dataframe.isnull().sum().reset_index()
    na_df.columns = ['Column', 'Number of NA']
    display(na_df.style.set_caption("Number of NA Values"))

def display_quantiles(dataframe):
    quantiles_df = dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T
    display(quantiles_df.style.format("{:.2f}").set_caption("Quantiles"))

def check_df(dataframe, head=5):
    print_section_title('Shape')
    print(dataframe.shape)
    print_section_title('Types')
    print(dataframe.dtypes.to_frame('Data Type').style.set_caption("Data Types"))
    print_section_title('Head & Tail')
    display_head_and_tail(dataframe, head)
    print_section_title('NA Values')
    display_na(dataframe)
    print_section_title('Quantiles')
    display_quantiles(dataframe)

check_df(df_3)

In [ ]:
df_3 = df_3.drop(columns=['Code'])

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.font_manager import FontProperties
import matplotlib.font_manager as fm
import koreanize_matplotlib  # (선택) 이 라이브러리를 쓰면 한글 폰트 자동 설정

# Colab에서 한글 폰트 설치 (이미 설치했으면 생략)
# !apt-get update -qq
# !apt-get install fonts-nanum -qq

# Seaborn, Matplotlib 설정
sns.set_theme(style="whitegrid")
plt.rcParams['axes.unicode_minus'] = False

# 원하는 폰트(예: NanumGothic) 사용
plt.rc('font', family='NanumGothic')

########################################
# 1. df_3 전처리: 2020년까지 필터링
########################################
# 예: df_3에 'Year' 열이 있고, 4개 재생에너지 열이 있다고 가정
df_3 = df_3[df_3['Year'] <= 2020].copy()

# 총 재생에너지 생산량(TWh) 계산
df_3['Total Renewable (TWh)'] = df_3[[
    'Electricity from wind (TWh)',
    'Electricity from hydro (TWh)',
    'Electricity from solar (TWh)',
    'Other renewables including bioenergy (TWh)'
]].sum(axis=1)

colors = sns.color_palette("husl")

########################################
# 2. 시각화
########################################
fig, ax = plt.subplots(figsize=(20, 8))

# 라인플롯
sns.lineplot(
    x='Year',
    y='Total Renewable (TWh)',
    data=df_3,
    marker='o',
    color=colors[0],
    ax=ax,
    label='Total Renewable Energy'
)

# 최대값 지점 찾아 주석 달기
max_point = df_3[df_3['Total Renewable (TWh)'] == df_3['Total Renewable (TWh)'].max()]
max_year = max_point['Year'].values[0]
max_val = max_point['Total Renewable (TWh)'].values[0]

ax.annotate(
    f"Max: {max_year}",
    xy=(max_year, max_val),
    xytext=(-50, 50),
    textcoords='offset points',
    arrowprops=dict(arrowstyle='->', color='black', lw=1),
    fontsize=12
)

# 그래프 제목, 축 라벨
plt.title('연도별 총 재생 에너지 생산량 (2020년까지)', fontsize=16, fontweight='bold')
plt.xlabel('Year', fontsize=12, fontweight='bold')
plt.ylabel('Total Renewable Energy (TWh)', fontsize=12, fontweight='bold')

# 그리드, 범례
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(fontsize=12)

# 최대값 주변(±1년) 강조 영역
ax.axvspan(max_year - 1, max_year + 1, alpha=0.2, color='yellow')

# 폰트 설정
font = FontProperties()
font.set_weight('bold')
ax.set_xticks(ax.get_xticks())
ax.set_yticks(ax.get_yticks())
ax.set_xticklabels(ax.get_xticks(), fontproperties=font)
ax.set_yticklabels(ax.get_yticks(), fontproperties=font)

# 축 라벨 각도, 크기
plt.xticks(rotation=45, fontsize=12, fontweight='bold')
plt.yticks(fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.font_manager import FontProperties
import matplotlib.font_manager as fm
import koreanize_matplotlib  # (선택) 이 라이브러리를 쓰면 한글 폰트 자동 설정

# Colab에서 한글 폰트 설치 (이미 설치했으면 생략)
# !apt-get update -qq
# !apt-get install fonts-nanum -qq

# Seaborn, Matplotlib 설정
sns.set_theme(style="whitegrid")
plt.rcParams['axes.unicode_minus'] = False

# 원하는 폰트(예: NanumGothic) 사용
plt.rc('font', family='NanumGothic')

energy_types = ['Electricity from wind (TWh)', 'Electricity from hydro (TWh)', 'Electricity from solar (TWh)', 'Other renewables including bioenergy (TWh)']

total_renewable_by_source_yearly = df_3.groupby('Year')[energy_types].sum().reset_index()

colors = ['dodgerblue', 'limegreen', 'gold', 'tomato']

plt.figure(figsize=(20, 8))

plt.stackplot(total_renewable_by_source_yearly['Year'],
              total_renewable_by_source_yearly['Electricity from wind (TWh)'],
              total_renewable_by_source_yearly['Electricity from hydro (TWh)'],
              total_renewable_by_source_yearly['Electricity from solar (TWh)'],
              total_renewable_by_source_yearly['Other renewables including bioenergy (TWh)'],
              labels=['풍력(TWh)', '수력(TWh)', '태양열(TWh)', '바이오에너지를 포함한 기타 재생 에너지(TWh)'], colors=colors)

plt.title('총 재생 에너지 생산량')
plt.xlabel('Year')
plt.ylabel('Production (TWh)')
plt.legend(loc='upper left', title='Energy Sources')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import koreanize_matplotlib

sns.set_theme(style="whitegrid", font="NanumGothic")
plt.rcParams['axes.unicode_minus'] = False

#############################################
# 1. CSV 파일 불러오기 및 전처리
#############################################
df = pd.read_csv('/content/03 modern-renewable-prod.csv')
df.columns = df.columns.str.strip()  # 컬럼명 앞뒤 공백 제거

# 주요 열: "Entity", "Year" 및 재생에너지 생산량 관련 열
energy_types = [
    'Electricity from wind (TWh)',
    'Electricity from hydro (TWh)',
    'Electricity from solar (TWh)',
    'Other renewables including bioenergy (TWh)'
]

# 숫자형 변환 및 결측치 처리
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
for col in energy_types:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    else:
        df[col] = 0

# 각 국가(Entity)별 최신 연도 데이터 선택
latest_data = df.sort_values('Year').groupby('Entity').last().reset_index()

# 각 국가에서 가장 많이 생산한 재생에너지 유형 결정
latest_data['Most Used Renewable Source'] = latest_data[energy_types].idxmax(axis=1)

# 색상 매핑 (풍력: 파란색, 수력: 초록, 태양광: 주황, 기타 재생: 빨강)
color_map = {
    'Electricity from wind (TWh)': 'blue',
    'Electricity from hydro (TWh)': 'green',
    'Electricity from solar (TWh)': 'orange',
    'Other renewables including bioenergy (TWh)': 'red'
}
latest_data['Color'] = latest_data['Most Used Renewable Source'].map(color_map)
latest_data['Color'] = latest_data['Color'].fillna('lightgrey')

#############################################
# 2. 세계 지도 데이터 불러오기 (대체 GeoJSON 사용)
#############################################
# datasets/geo-countries의 GeoJSON 파일 사용 (국가 이름은 'ADMIN'에 있음)
url_geojson = "https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson"
world = gpd.read_file(url_geojson)
# 'ADMIN' 열을 'name'으로 변경
world.rename(columns={'ADMIN': 'name'}, inplace=True)

#############################################
# 3. 국가 이름 매핑 및 데이터 병합
#############################################
# 국가 이름이 다를 수 있으므로 치환 (예: "United States of America" -> "United States")
name_mapping = {
    "United States of America": "United States",
    "Russian Federation": "Russia",
    "Republic of Korea": "South Korea",
    "Korea": "South Korea",
    "Iran (Islamic Republic of)": "Iran",
    # 필요에 따라 추가
}
world['name'] = world['name'].replace(name_mapping)

# 최신 재생에너지 생산 데이터의 'Entity'와 세계 지도 'name'을 기준으로 병합
merged = world.set_index('name').join(latest_data.set_index('Entity'))
merged['Color'] = merged['Color'].fillna('lightgrey')

#############################################
# 4. 지도 시각화 및 범례 추가
#############################################
def add_legend(ax):
    legend_labels = {
        'blue': '풍력',
        'green': '수력',
        'orange': '태양광',
        'red': '기타 재생에너지',
        'lightgrey': '데이터 없음'
    }
    y_offset = 0.02
    for color, label in legend_labels.items():
        ax.annotate(
            label,
            xy=(1.02, 1 - y_offset),
            xycoords='axes fraction',
            fontsize=10,
            color=color,
            weight='bold'
        )
        y_offset += 0.05

fig, ax = plt.subplots(1, 1, figsize=(15, 10))
merged.plot(color=merged['Color'], ax=ax, legend=False)
add_legend(ax)
ax.set_title('국가별 가장 많이 생산한 재생에너지 유형)', fontsize=16, fontweight='bold')
ax.set_axis_off()

plt.show()


print('그래프 분석: 이 지리 공간 시각화는 최신 데이터를 기반으로 각 국가에서 가장 많이 사용되는 재생 에너지원을 매핑합니다.')
print('이 지도는 각 지역에서 주로 사용되는 재생 에너지 유형을 나타내도록 색상으로 구분되어 있습니다: 파란색은 풍력, 녹색은 수력, 주황색은 태양광, 빨간색은 기타 재생 에너지원입니다.')
print('데이터가 없거나 데이터가 부족한 국가는 연한 회색으로 표시되어 있습니다. ')
print('이 시각화 도구는 지리적 추세와 지역 의존성을 강조하면서 재생 에너지 선호도에 대한 글로벌 관점을 제공합니다.')
print('이 도구는 다양한 에너지 환경을 강조하고 특정 유형의 재생 에너지에서 성장 가능성이 있는 지역을 식별하는 데 도움이 됩니다.')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

# Seaborn 및 폰트 설정
sns.set_theme(style="whitegrid", font="NanumGothic")
sns.set_context("talk")  # 'paper', 'notebook', 'talk', 'poster' 중 선택
plt.rcParams['axes.unicode_minus'] = False

# 그래프 크기 설정
fig, ax1 = plt.subplots(figsize=(12, 8))

# 색상 코드(빨간색/파란색 계열)
color_co2 = "#d62728"  # 빨간색
color_lpi = "#1f77b4"  # 파란색

# ----------------------------
# 왼쪽 y축: CO₂ 배출량
# ----------------------------
ax1.set_xlabel('연도', fontsize=14, fontweight='bold')
ax1.set_ylabel('CO₂ 배출량(톤)', fontsize=14, fontweight='bold', color=color_co2)

line1 = ax1.plot(
    merged['Year'],
    merged['CO₂ 배출량(톤)'],
    marker='o',
    markersize=6,
    linewidth=2,
    color=color_co2,
    label='CO₂ 배출량'
)

ax1.tick_params(axis='y', labelcolor=color_co2)
ax1.tick_params(axis='x', rotation=45, labelsize=12)
ax1.grid(True, linestyle='--', alpha=0.7)

# ----------------------------
# 오른쪽 y축: Living Planet Index (LPI)
# ----------------------------
ax2 = ax1.twinx()
ax2.set_ylabel('생물다양성 지표 (LPI)', fontsize=14, fontweight='bold', color=color_lpi)

line2 = ax2.plot(
    merged['Year'],
    merged['Living Planet Index'],
    marker='s',
    markersize=6,
    linewidth=2,
    color=color_lpi,
    label='LPI'
)

ax2.tick_params(axis='y', labelcolor=color_lpi)
ax2.tick_params(axis='x', labelsize=12)

# ----------------------------
# 범례 병합
# ----------------------------
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='best', fontsize=12)

# ----------------------------
# 그래프 제목 및 레이아웃
# ----------------------------
plt.title('1970~2020 탄소 배출량 vs. 생물다양성 (LPI)', fontsize=18, fontweight='bold')
plt.tight_layout()
plt.show()

지리적

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/co2_data.csv')
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df_clean = df.dropna(subset=['co2_per_capita', 'co2_growth_prct', 'co2'])

In [ ]:
df_clean.loc[:, 'population'] = df_clean['population'].bfill()

In [ ]:
print(df_clean[['co2_per_capita', 'co2_growth_prct', 'co2']].head())

In [ ]:
df_clean.isnull().sum()

In [ ]:
island_countries = [
    # 아메리카
    "Grenada", "Dominica", "Barbados", "Bahamas", "Saint Lucia",
    "Saint Vincent and the Grenadines", "Saint Kitts and Nevis", "Haiti", "Antigua and Barbuda",
    "Jamaica", "Cuba", "Trinidad and Tobago", "Puerto Rico", "British Virgin Islands",
    "U.S. Virgin Islands",

    # 아시아
    "Timor-Leste", "Maldives", "Bahrain", "Brunei", "Sri Lanka", "Singapore",
    "Indonesia", "Japan", "Taiwan", "Philippines", "British Indian Ocean Territory",

    # 아프리카
    "Madagascar", "Mauritius", "Sao Tome and Principe", "Seychelles", "Cape Verde", "Comoros",
    "Saint Helena", "Ascension Island", "Tristan da Cunha",

    # 오세아니아
    "Nauru", "New Zealand", "Marshall Islands", "Micronesia", "Vanuatu", "Samoa", "Solomon Islands",
    "Kiribati", "Tonga", "Tuvalu", "Papua New Guinea", "Palau", "Fiji", "Cook Islands",

    # 유럽
    "United Kingdom", "Ireland", "Malta", "Iceland", "Cyprus",

    # 기타 섬 지역
    "Montserrat", "Turks and Caicos Islands", "Cayman Islands", "Curaçao",
    "Saint Barthelemy", "Saint Martin", "Saint Pierre and Miquelon", "Faroe Islands",
    "Gibraltar", "Isle of Man", "Guernsey", "Jersey", "Madeira", "Greenland",
    "New Caledonia", "Niue", "Norfolk Island", "Pitcairn Islands", "Tokelau",
    "Wallis and Futuna", "Reunion", "Christmas Island", "Cocos (Keeling) Islands"
]




# df['island']에 섬나라 여부를 할당
df_clean = df_clean.copy()  # 데이터프레임을 복사
df_clean.loc[:, 'island'] = df_clean['country'].apply(lambda x: "Island" if x in island_countries else "Continent")


In [ ]:
df_clean['island'].head()

In [ ]:
lat = pd.read_csv('/content/lat.csv')
lat.head()

In [ ]:
print(df_clean['country'].unique()), print(lat['country'].unique())

In [ ]:
# 국가명 매핑 테이블 (모든 국가명 통일)
country_mapping = {
    "Cote d'Ivoire": "Côte d'Ivoire",
    "Russian Federation": "Russia",
    "United States": "United States of America",
    "South Korea": "Korea, Republic of",
    "Iran": "Iran, Islamic Republic of",
    "Venezuela": "Venezuela, Bolivarian Republic of",
    "Bolivia": "Bolivia, Plurinational State of",
    "North Macedonia": "Macedonia, the former Yugoslav Republic of",
    "Tanzania": "Tanzania, United Republic of",
    "Vietnam": "Viet Nam",
    "North Korea": "Korea, Democratic People's Republic of",
    "Czechia": "Czech Republic",
    "Libya": "Libyan Arab Jamahiriya",
    "Laos": "Lao People's Democratic Republic",
    "Taiwan": "Taiwan, Province of China",
    "Kuwait": "Kuwait",
    "Syria": "Syrian Arab Republic",
    "Moldova": "Moldova, Republic of",
    "Kosovo": "Kosovo",
    "Montenegro": "Montenegro",
    "Palestine": "Palestinian Territory, Occupied"
}



# 국가명 정리
df_clean['country'] = df_clean['country'].replace(country_mapping)
lat['country'] = lat['country'].replace(country_mapping)

# 대륙 및 그룹명 제외
continents_groups = ["Africa", "Asia", "Europe", "North America", "South America", "Oceania",
                     "World", "European Union (27)", "European Union (28)", "High-income countries",
                     "Low-income countries", "Lower-middle-income countries", "Upper-middle-income countries"]
df_clean = df_clean[~df_clean['country'].isin(continents_groups)]

# 병합
df_clean = df_clean.merge(lat[['country', 'lat_min']], on='country', how='left')

In [ ]:
cold = [
    "Russia",
    "Canada",
    "Alaska",
    "Sweden",
    "Finland",
    "Norway",
    "Iceland",
    "Greenland"
]

# df['region'] 컬럼에 cold, notcold 구분하여 할당
df_clean['climate'] = df_clean['country'].apply(lambda x: 'cold' if x in cold else 'notcold')

df_clean.head()

In [ ]:
# 북반구/남반구 구분
df_clean['hemisphere'] = df_clean['lat_min'].apply(lambda x: 'northern' if x >= 0 else 'southern')

# 결과 확인
print(df_clean[['country', 'lat_min', 'hemisphere']].sample(10))

In [ ]:
# CSV 파일로 저장
df_clean.to_csv('final.csv', index=False)

In [ ]:
import pandas as pd

# CSV 파일 로드
data = pd.read_csv("owid-co2-data.csv")

# 기준 연도를 1950년대~2023년 사이로 설정
data = data[(data['year'] >= 1950) & (data['year'] <= 2023)]

# 필요한 열만 선택
data = data[['country', 'year', 'co2']]

# 특정 국가 목록 제외
excluded_countries = [
    "World",
    "Asia",
    "Asia (excl. China & India)",
    "Africa",
    "Europe",
    "Europe (excl. EU-27)",
    "Europe (excl. EU-28)",
    "North America",
    "North America (excl. USA)",
    "South America",
    "Oceania",
    "European Union (27)",
    "European Union (28)",
    "High-income countries",
    "Upper-middle-income countries",
    "Lower-middle-income countries",
    "Low-income countries",
    "International transport",
    "Non-OECD (GCP)",
    "Asia (GCP)",
    "Europe (GCP)",
    "North America (GCP)",
    "South America (GCP)",
    "Oceania (GCP)",
    "OECD (GCP)",
    "Asia (excl. China and India)",
    "Middle East (GCP)",
    "Africa (GCP)",
    "International shipping",
    "International aviation"
]
data = data[~data['country'].isin(excluded_countries)]

# 국가별 1950~2024년 CO₂ 배출량 합산
total_co2_by_country = data.groupby('country')['co2'].sum().reset_index()

# CO₂ 배출량이 가장 많은 10개 국가 선택
top_20_countries = total_co2_by_country.sort_values(by='co2', ascending=False).head(20)

# 결과 출력
print(top_20_countries)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# CSV 파일 로드
data = pd.read_csv("owid-co2-data.csv")

# 기준 연도를 1950년대~2023년 사이로 설정
data = data[(data['year'] >= 1950) & (data['year'] <= 2023)]

# 필요한 열만 선택
data = data[['country', 'year', 'co2']]

# 특정 국가 목록 제외
excluded_countries = [
    "World",
    "Asia",
    "Asia (excl. China & India)",
    "Africa",
    "Europe",
    "Europe (excl. EU-27)",
    "Europe (excl. EU-28)",
    "North America",
    "North America (excl. USA)",
    "South America",
    "Oceania",
    "European Union (27)",
    "European Union (28)",
    "High-income countries",
    "Upper-middle-income countries",
    "Lower-middle-income countries",
    "Low-income countries",
    "International transport",
    "Non-OECD (GCP)",
    "Asia (GCP)",
    "Europe (GCP)",
    "North America (GCP)",
    "South America (GCP)",
    "Oceania (GCP)",
    "OECD (GCP)",
    "Asia (excl. China and India)",
    "Middle East (GCP)",
    "Africa (GCP)",
    "International shipping",
    "International aviation"
]
data = data[~data['country'].isin(excluded_countries)]

# 국가별 1950~2024년 CO₂ 배출량 합산
total_co2_by_country = data.groupby('country')['co2'].sum().reset_index()

# CO₂ 배출량이 가장 많은 20개 국가 선택
top_20_countries = total_co2_by_country.sort_values(by='co2', ascending=False).head(20)

# 상위 20개 국가의 데이터만 필터링
top_20_data = data[data['country'].isin(top_20_countries['country'])]

# Upper-middle-income countries 목록
upper_middle_income_countries = [
    "India", "Mexico", "South Africa", "Brazil",
    "Indonesia", "Iran", "China", "Saudi Arabia"
]

# High-income countries 목록
high_income_countries = [
    "United States", "Russia", "Germany", "Japan",
    "United Kingdom", "Canada", "Ukraine", "France",
    "Italy", "Poland", "South Korea", "Australia"
]

# Upper-middle-income countries 데이터만 필터링
upper_middle_income_data = data[data['country'].isin(upper_middle_income_countries)]

# High-income countries 데이터만 필터링
high_income_data = data[data['country'].isin(high_income_countries)]

# Upper-middle-income countries 그래프 그리기
plt.figure(figsize=(12, 8))
for country in upper_middle_income_countries:
    country_data = upper_middle_income_data[upper_middle_income_data['country'] == country]
    plt.plot(country_data['year'], country_data['co2'], label=country)

# 2004년 기준선 추가
plt.axvline(x=2004, color='r', linestyle='--', label='2004')

plt.title("CO2 Emissions by Upper-Middle-Income Countries (1950-2023)", fontsize=16)
plt.xlabel("Year", fontsize=12)
plt.ylabel("CO2 Emissions (metric tons)", fontsize=12)
plt.legend(title="Countries", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

# High-income countries 그래프 그리기
plt.figure(figsize=(12, 8))
for country in high_income_countries:
    country_data = high_income_data[high_income_data['country'] == country]
    plt.plot(country_data['year'], country_data['co2'], label=country)

# 2004년 기준선 추가
plt.axvline(x=2004, color='r', linestyle='--', label='2004')

plt.title("CO2 Emissions by High-Income Countries (1950-2023)", fontsize=16)
plt.xlabel("Year", fontsize=12)
plt.ylabel("CO2 Emissions (metric tons)", fontsize=12)
plt.legend(title="Countries", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

-----


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt
# 한글화
!pip install koreanize-matplotlib
import koreanize_matplotlib

In [ ]:
# 데이터 출처 : https://github.com/owid/co2-data?tab=readme-ov-file

df = pd.read_csv('/content/owid-co2-data.csv')
df

In [ ]:
#데이터프레임 기본 구조 확인
df_info = df.info()

# 결측치 확인
missing_values = df.isnull().sum()

df_info, missing_values

In [ ]:
# 결측치 처리
# 시계열 데이터를 고려한 backward fill 적용 ('gdp', 'population' 컬럼)
df[['gdp', 'population']] = df[['gdp', 'population']].bfill()

# backward fill 적용 후 결측치 재확인
filled_missing_values = df[['gdp', 'population']].isnull().sum()

filled_missing_values

남아 있는 결측치는 Zimbabwe의 2023년도 데이터

In [ ]:
#'iso_code' 컬럼의 결측치 중 'country' 컬럼에 (GCP)가 포함되어 있는 데이터를 GCP로 처리
df.loc[df['iso_code'].isnull() & df['country'].str.contains('GCP'), 'iso_code'] = 'GCP'

# 처리 후 결과 확인
iso_code_missing_after = df['iso_code'].isnull().sum()
iso_code_value_counts = df['iso_code'].value_counts()

iso_code_missing_after, iso_code_value_counts.head(10)

In [ ]:
# 전세계(world) 연도별 탄소배출량 확인
# 필요한 컬럼만 선택
co2_columns = ["country", "year", "co2", "co2_growth_prct"]
df_co2 = df[co2_columns].fillna(0) # 결측치를 0으로 대체

# 전 세계 데이터만 필터링
df_global = df_co2[df_co2["country"] == "World"]

# 데이터 확인
print(df_global.tail())  # 최근 데이터 확인

In [ ]:
# 그래프 설정
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_global, x="year", y="co2", marker="o", color="b")

# 그래프 스타일 설정
plt.title("전 세계 연도별 CO₂ 배출량 변화 (1970~2023)", fontsize=14)
plt.xlabel("연도", fontsize=12)
plt.ylabel("CO₂ 배출량 (백만 톤)", fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
# 최근 10년 데이터 필터링 (2013~2023)
df_recent = df_global[df_global["year"] >= 2013]

# 최근 10년 연평균 Co2 감소율 계산
avg_reduction_rate = df_recent["co2_growth_prct"].mean()

# 결과 출력
print(f"최근 10년간 연평균 CO₂ 변화율: {avg_reduction_rate:.2f}%")

In [ ]:
## 2050년까지 탄소배출량 예측
# 2050년까지 연도 생성
future_years = np.arange(2023, 2051)

# 현재 감축률 적용하여 미래 예상 Co2 배출량 계산
future_co2 = [df_global["co2"].iloc[-1]]  # 최신 Co2 값 가져오기
for _ in future_years[1:]:
    future_co2.append(future_co2[-1] * (1 + avg_reduction_rate / 100))  # 연평균 감소율 적용

# 데이터프레임 생성
df_future = pd.DataFrame({"year": future_years, "predicted_co2": future_co2})

# 예측 결과 시각화
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_global, x="year", y="co2", label="실제 Co2 배출량", color="b")
sns.lineplot(data=df_future, x="year", y="predicted_co2", label="예측 Co2 배출량", color="r", linestyle="dashed")

# 그래프 스타일 설정
plt.axhline(y=0, color="black", linestyle="--", label="탄소중립 목표 (0)")
plt.title("2050년 Co2 배출량 예측 (현재 감축률 적용)", fontsize=14)
plt.xlabel("연도", fontsize=12)
plt.ylabel("Co2 배출량 (백만 톤)", fontsize=12)
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 필요한 연평균 감축률 계산 (현재 Co2 배출량 → 2050년 0 목표)
current_co2 = df_global["co2"].iloc[-1]  # 최신 Co2 값
years_left = 2050 - 2023  # 남은 기간

# 감축 공식: (1 - 감축률)^years_left = 목표값 / 현재값
required_reduction_rate = 1 - (0 / current_co2) ** (1 / years_left)

# 결과 출력
print(f"2050년 탄소중립을 달성하려면 연평균 {required_reduction_rate*100:.2f}%의 감축률이 필요함.")

In [ ]:
# 온실가스별로 기후에 끼치는 영향 확인하기
# 온실가스 배출 및 기후 변화 영향 컬럼 선택
ghg_climate_columns = [
    "temperature_change_from_co2", "temperature_change_from_ghg", "temperature_change_from_ch4", "temperature_change_from_n2o"
]

# 전 세계 데이터 선택 (World)
world_ghg_data = df[df["country"] == "World"]

# 기온 변화를 나타내는 시계열 그래프
plt.figure(figsize=(12, 6))
plt.plot(world_ghg_data["year"], world_ghg_data["temperature_change_from_co2"], label="Temperature Change from CO2", linestyle='-', marker='o')
plt.plot(world_ghg_data["year"], world_ghg_data["temperature_change_from_ghg"], label="Temperature Change from GHG", linestyle='-', marker='s')
plt.plot(world_ghg_data["year"], world_ghg_data["temperature_change_from_ch4"], label="Temperature Change from CH4", linestyle='-', marker='^')
plt.plot(world_ghg_data["year"], world_ghg_data["temperature_change_from_n2o"], label="Temperature Change from N2O", linestyle='-', marker='D')

plt.xlabel("연도")
plt.ylabel("Temperature Change (°C)")
plt.title("온실가스로 인한 기후 변화")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 국가 리스트 정의
developing = ["India", "Mexico", "South Africa", "Brazil", "Indonesia", "Iran", "China", "Saudi Arabia"]
developed = ["United States", "Russia", "Germany", "Japan", "United Kingdom",
             "Canada", "Ukraine", "France", "Italy", "Poland", "South Korea", "Australia"]

# 1985년 이후 데이터 필터링
df_filtered = df[df['year'] > 1985]

# 선진국 데이터 pivot
df_developed = df_filtered[df_filtered['country'].isin(developed)].pivot(index='country', columns='year', values='co2_per_unit_energy')

# 개발도상국 데이터 pivot
df_developing = df_filtered[df_filtered['country'].isin(developing)].pivot(index='country', columns='year', values='co2_per_unit_energy')

# 선진국 그래프
plt.figure(figsize=(14, 6))
for country in df_developed.index:
    plt.plot(df_developed.columns, df_developed.loc[country], marker='o', label=country)

plt.xlabel('Year')
plt.ylabel('CO2 per Unit Energy')
plt.title('선진국 : 탄소배출집약도 (1986~)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

# 개발도상국 그래프
plt.figure(figsize=(14, 6))
for country in df_developing.index:
    plt.plot(df_developing.columns, df_developing.loc[country], marker='o', label=country)

plt.xlabel('Year')
plt.ylabel('CO2 per Unit Energy')
plt.title('개발도상국 : 탄소배출집약도 (1986~)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

# 선진국과 개발도상국 합쳐진 그래프
plt.figure(figsize=(14, 8))
for country in df_developed.index:
    plt.plot(df_developed.columns, df_developed.loc[country], marker='o', label=f"Developed: {country}")

for country in df_developing.index:
    plt.plot(df_developing.columns, df_developing.loc[country], linestyle='--', marker='x', label=f"Developing: {country}")

plt.xlabel('Year')
plt.ylabel('CO2 per Unit Energy')
plt.title('선진국 vs 개발도상국 : 탄소배출집약도 (1986~)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# 결측치 처리 (보간법 + 중앙값 대체 + 0 대체)
df_cleaned['gdp'] = df_cleaned.groupby('country')['gdp'].transform(lambda x: x.ffill().bfill())
df_cleaned['population'] = df_cleaned.groupby('country')['population'].transform(lambda x: x.ffill().bfill())
df_cleaned['gdp'] = df_cleaned.groupby('country')['gdp'].transform(lambda x: x.fillna(x.median()))
df_cleaned['co2_growth_prct'] = df_cleaned.groupby('country')['co2_growth_prct'].transform(lambda x: x.fillna(x.median()))

# 이상치 탐색/처리
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] < lower_bound) | (df[column] > upper_bound)]

def replace_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])
    df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])
    return df

key_columns = ['gdp', 'population', 'co2', 'co2_growth_prct', 'coal_co2', 'oil_co2', 'gas_co2']
for col in key_columns:
    if col in df_cleaned.columns:
        df_cleaned = replace_outliers(df_cleaned, col)

# 전처리된 데이터 저장
output_filename = "processed_owid_co2_data.csv"
df_cleaned.to_csv(output_filename, index=False)

# 파일 다운로드
from google.colab import files
files.download(output_filename)

print(f"전처리 완료! 파일이 저장되었습니다: {output_filename}")